In [97]:
#Use this to extract specific information via Web Scraping.
#Not reliable to NLP purposes because scraped data contains lots of web traffic & advertisements. 
import sys, os
import glob
cwd = os.getcwd()
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer 
from collections import Counter
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import reuters, stopwords
import re
import nltk
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt

In [98]:
finviz_url = 'https://finviz.com/quote.ashx?t='
tickers = ['AMZN', 'GOOG', 'TSLA']
news_tables = {}
for ticker in tickers:
    url = finviz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(req)
    html = BeautifulSoup(response, features='html.parser')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

parsed_data = []
url_dict = {}
for ticker, news_table in news_tables.items():
    url_ls = []
    for row in news_table.findAll('tr'):

        if row.a is None:
            pass
        else:
            title = row.a.text
            date_data = row.td.text.split(' ')
            url_extraction = str(row)
            string_search = re.search('https(.+?)"', url_extraction)
            if string_search:
                extracted_url = string_search.group(1)
                final_url = 'http' + extracted_url
                url_ls.append(final_url)


        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]

        parsed_data.append([ticker, date, time, title])

    ticker_url_list = pd.DataFrame(url_ls, columns = [ticker])
    url_dict[ticker] = ticker_url_list
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])

vader = SentimentIntensityAnalyzer()

f = lambda title: vader.polarity_scores(title)['compound']
df['compound'] = df['title'].apply(f)
df['date'] = pd.to_datetime(df.date).dt.date


In [99]:
item='AMZN'
testing = url_dict['AMZN']
list = testing[item].tolist()
for item in list:
    print(list)

['http://finance.yahoo.com/news/u-begins-antitrust-review-amazon-011542610.html', 'http://finance.yahoo.com/news/u-begins-antitrust-review-amazon-011104249.html', 'http://www.wsj.com/articles/ftc-investigating-amazon-deal-to-buy-one-medical-network-of-health-clinics-11662157363?siteid=yhoof2', 'http://finance.yahoo.com/news/amazon-amzn-stock-moves-0-214509401.html', 'http://www.barrons.com/articles/dan-niles-satori-fund-gambling-apple-stocks-51662069936?siteid=yhoof2', 'http://finance.yahoo.com/news/u-ftc-asks-amazon-one-211317136.html', 'http://www.fool.com/investing/2022/09/02/why-plug-power-shares-plunged-40-on-friday/?source=eptyholnk0000202&amp;utm_source=yahoo-host&amp;utm_medium=feed&amp;utm_campaign=article', 'http://finance.yahoo.com/video/rings-power-viewers-report-issues-201822535.html', 'http://finance.yahoo.com/news/amazon-closes-abandons-plans-dozens-155259786.html', 'http://www.fool.com/investing/2022/09/02/why-cheetah-stock-soared-friday-morning/?source=eptyholnk0000202

In [100]:
top_record_count=20
for item in url_dict:
    url_df = (url_dict[item])
    ticker_url_list = url_df[item].tolist()
    for url in ticker_url_list:
        req = Request(url=url, headers={'user-agent': 'my-app'})
        response = urlopen(req)
        html = BeautifulSoup(response, features='html.parser')
        raw_text = html.get_text()
        lines = (line.strip() for line in raw_text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

KeyboardInterrupt: 

In [104]:
raw_text

"\n\n\n\n\n\n      Why a Bearish Money Manager Likes Gambling Stocks and Is Ready to Dump Apple | Barron's\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n \n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n    We've detected you are on Internet Explorer. For the best Barrons.com experience, please update to a modern browser.\n    \n\nCHROME\nSAFARI\nFIREFOX\n\n\n\n\n \n\n\nWe've detected you are on Internet Explorer. For the best Barrons.com experience, please update to a modern browser.GoogleFirefoxBarron'sSubscribe NowSign In\n \n\n\n\n\n\n \n\n\n\nBarrons\n\n\n\n\nNext:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nShare\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nThis copy is for your personal, non-commercial use only. To order presentation-ready copies for distribution to your colleagues, client

In [101]:
sample_url = str(text)
strip_lines=sample_url.strip()

lemmatizer = WordNetLemmatizer()
stop_words_list = set(stopwords.words('english'))
# stop_words_list = list(nltk_stop_words)
include_in_stopwords = ['ha']
remove_from_stopwords = ['all', 'below', 'being', 'too']
stop_words = [elem for elem in stop_words_list if elem not in remove_from_stopwords]
stop_words = stop_words+include_in_stopwords
regex = re.compile("[^a-zA-Z ]")
re_clean = regex.sub('', strip_lines)
words = word_tokenize(re_clean)
lem = [lemmatizer.lemmatize(word) for word in words]
output = [c for c in lem if c.islower()]
output = [word.lower() for word in output if word.lower() not in stop_words]
top_10 = dict(Counter(output).most_common(top_record_count))
# top_10_df = pd.DataFrame(list(top_10.items()), columns=['word', 'count'])

In [103]:
strip_lines

"Why a Bearish Money Manager Likes Gambling Stocks and Is Ready to Dump Apple | Barron's\nWe've detected you are on Internet Explorer. For the best Barrons.com experience, please update to a modern browser.\nCHROME\nSAFARI\nFIREFOX\nWe've detected you are on Internet Explorer. For the best Barrons.com experience, please update to a modern browser.GoogleFirefoxBarron'sSubscribe NowSign In\nBarrons\nNext:\nShare\nThis copy is for your personal, non-commercial use only. To order presentation-ready copies for distribution to your colleagues, clients or customers visit http://www.djreprints.com.\nhttps://www.barrons.com/articles/dan-niles-satori-fund-gambling-apple-stocks-51662069936\nMarkets\nQ&A\nWhy a Bearish Money Manager Likes Gambling Stocks and Is Ready to Dump Apple\nBy\nEric J. Savitz\nUpdated Sept. 2, 2022 5:31 pm ET / Original Sept. 2, 2022 12:30 am ET\nOrder Reprints\nPrint Article\nDaniel Niles runs the Satori Fund.\nPhotograph by Gabriela Hasbun\nDan Niles thinks the stock mar

In [102]:
url

'http://finance.yahoo.com/news/u-ftc-asks-amazon-one-211317136.html'

In [ ]:
html = BeautifulSoup(response, features='html.parser')
text = html.get_text()

IncompleteRead: IncompleteRead(14268 bytes read)

In [ ]:
lines = (line.strip() for line in raw_text.splitlines())
text = '.'.split(lines)
text

TypeError: must be str or None, not generator

In [ ]:
lines

<generator object <genexpr> at 0x000002D928E4B120>